In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [ ]:
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt").toDF()
data = MLUtils.convertVectorColumnsToML(data)

In [ ]:
data.show(5)

## Evaluation

### Binary Classification

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import BinaryLogisticRegressionSummary

trainingData, testData = data.randomSplit([0.7, 0.3])

logr = LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

logrModel = logr.fit(trainingData)

print("Weights: %s Intercept: %s" % (logrModel.coefficients, logrModel.intercept))

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictionsLogR = logrModel.transform(testData)

evaluator = BinaryClassificationEvaluator().setLabelCol("label") \
                            .setRawPredictionCol("rawPrediction") \
                            .setMetricName("areaUnderROC")

roc = evaluator.evaluate(predictionsLogR)
print(roc)

### Multiclass Classification

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, IndexToString, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestClassificationModel

labelIndexer = StringIndexer().setInputCol("label") \
                                .setOutputCol("indexedLabel").fit(data)

labelConverter = IndexToString().setInputCol("prediction") \
                                .setOutputCol("predictedLabel") \
                                .setLabels(labelIndexer.labels)

featureIndexer = VectorIndexer().setInputCol("features") \
                                .setOutputCol("indexedFeatures") \
                                .setMaxCategories(4).fit(data)

rfC = RandomForestClassifier().setLabelCol("indexedLabel") \
                                .setFeaturesCol("indexedFeatures") \
                                .setNumTrees(3)
        
pipelineRFC = Pipeline().setStages([labelIndexer, featureIndexer, rfC, labelConverter])

modelRFC = pipelineRFC.fit(trainingData)

predictionsRFC = modelRFC.transform(testData)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator().setLabelCol("indexedLabel") \
                                        .setPredictionCol("prediction") \
                                        .setMetricName("accuracy")

accuracy = evaluator.evaluate(predictionsRFC)

print("Test Error = %s" % (1.0 - accuracy))

### Regression

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import RandomForestRegressionModel

rfR = RandomForestRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures")

pipelineRFR = Pipeline().setStages([featureIndexer, rfR])

modelRFR = pipelineRFR.fit(trainingData)

predictionsRFR = modelRFR.transform(testData)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator().setLabelCol("label") \
                                .setPredictionCol("prediction") \
                                .setMetricName("rmse")

rmse = evaluator.evaluate(predictionsRFR)

print("Root Mean Squared Error (RMSE) = %s" % rmse)

### Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

logr = LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

logrModel = logr.fit(trainingData)

print("Weights: %s Intercept: %s" % (logrModel.coefficients, logrModel.intercept))

In [ ]:
trainingSummaryLR = logrModel.summary
trainingSummaryLR.areaUnderROC

In [ ]:
fMeasure = trainingSummaryLR.fMeasureByThreshold

fMeasure.show(3)

In [ ]:
from pyspark.sql import functions as F

maxFMeasure = fMeasure.agg({"F-Measure": "max"}).head()[0]
print(maxFMeasure)
maxFMeasure = fMeasure.agg(F.max(F.col("F-Measure"))).head()[0]
print(maxFMeasure)

bestThreshold = fMeasure.where(F.col("F-Measure") == maxFMeasure).select("threshold").head()[0]
print(bestThreshold)

In [ ]:
trainingSummaryLR.pr.show(3)
trainingSummaryLR.precisionByThreshold.show(3)

In [ ]:
trainingSummaryLR.recallByThreshold.show(3)
trainingSummaryLR.roc.show(3)

### Linear Regression

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

lrModel = lr.fit(trainingData)

print("Weights: %s Intercept: %s" % (lrModel.coefficients, lrModel.intercept))

In [ ]:
trainingSummaryLLS = lrModel.summary

print(trainingSummaryLLS.explainedVariance)

print(trainingSummaryLLS.meanAbsoluteError)

print(trainingSummaryLLS.meanSquaredError)

print(trainingSummaryLLS.r2)

In [ ]:
trainingSummaryLLS.residuals.show(3)

print(trainingSummaryLLS.rootMeanSquaredError)

In [ ]:
sc.stop()